In [1]:
import json,folium
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(3)
pop = pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
0,도,경기도,13589432,0.104
1,시군,경기도 가평군,62197,0.057
2,읍면동,경기도 가평군 가평읍,19555,0.067
3,읍면동,경기도 가평군 북면,3815,0.032
4,읍면동,경기도 가평군 상면,5669,0.037


In [4]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
8,시군,경기도 고양시,1075202,0.106
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102


In [5]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102
56,시군,경기도 과천시,77775,0.112


In [6]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [7]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율,시군구
1,시군,경기도 가평군,62197,0.057,가평군
9,구,경기도 고양시 덕양구,487874,0.107,고양시덕양구
31,구,경기도 고양시 일산동구,296590,0.110,고양시일산동구
44,구,경기도 고양시 일산서구,290738,0.102,고양시일산서구
56,시군,경기도 과천시,77775,0.112,과천시


In [8]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.057
고양시덕양구,구,경기도 고양시 덕양구,487874,0.107
고양시일산동구,구,경기도 고양시 일산동구,296590,0.110
고양시일산서구,구,경기도 고양시 일산서구,290738,0.102
과천시,시군,경기도 과천시,77775,0.112


In [9]:
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.057
고양시덕양구,구,경기도 고양시 덕양구,487874,0.107
고양시일산동구,구,경기도 고양시 일산동구,296590,0.110
고양시일산서구,구,경기도 고양시 일산서구,290738,0.102
과천시,시군,경기도 과천시,77775,0.112


In [10]:
gpark = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')
gpark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17525 entries, 0 to 17524
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          17525 non-null  object 
 1   공원명           17525 non-null  object 
 2   공원구분          17525 non-null  object 
 3   소재지도로명주소      3692 non-null   object 
 4   소재지지번주소       17335 non-null  object 
 5   위도            17513 non-null  float64
 6   경도            17514 non-null  float64
 7   공원면적          17525 non-null  float64
 8   공원보유시설(운동시설)  4821 non-null   object 
 9   공원보유시설(유희시설)  6200 non-null   object 
 10  공원보유시설(편익시설)  4842 non-null   object 
 11  공원보유시설(교양시설)  805 non-null    object 
 12  공원보유시설(기타시설)  2577 non-null   object 
 13  지정고시일         14451 non-null  object 
 14  관리기관명         16033 non-null  object 
 15  전화번호          16343 non-null  object 
 16  데이터기준일자       17525 non-null  object 
 17  제공기관코드        17525 non-null  object 
 18  제공기관명         17525 non-nu

In [11]:
gpark.head(3)

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구
1,36110-00058,푸름샘 소공원(소1-1),소공원,세종특별자치시 마음안로 161,NaN,36.516117,127.233781,1820.0,NaN,NaN,앉음벽,NaN,실개천,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
2,36110-00059,솔숲 소공원(소1-2),소공원,세종특별자치시 만남1북로 73,NaN,36.508995,127.230040,1037.0,NaN,조합놀이대+ 그네+ 시소+ 흔들놀이,의자+ 파고라,NaN,안내판,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시


In [12]:
gp = gpark[['공원명','공원면적','소재지지번주소','경도', '위도']]
gp.head()

,공원명,공원면적,소재지지번주소,경도,위도
0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,푸름샘 소공원(소1-1),1820.0,NaN,127.233781,36.516117
2,솔숲 소공원(소1-2),1037.0,NaN,127.230040,36.508995
3,마디꽃 수변공원(수1-1),4928.0,NaN,127.257316,36.495962
4,꽃 수변공원(수2-3),3575.0,NaN,127.254482,36.473133


In [13]:
gp.dropna(subset=['소재지지번주소'], inplace=True)
gp.isna().sum() 

공원명         0
공원면적        0
소재지지번주소     0
경도         11
위도         12
dtype: int64

In [14]:
gp['경도'].head(20).tail(10)

32    126.977273
33    126.970926
34    126.965898
35    126.981103
36    126.970123
37    126.969601
38    126.969201
39    126.991493
40    127.012481
41    127.013882
Name: 경도, dtype: float64

In [15]:
gp[gp['경도'].isnull()]

,공원명,공원면적,소재지지번주소,경도,위도
9148,호돌이,1545.1,서울특별시 송파구 문정동 150-11,NaN,37.487446
9963,(신청사)어린이공원 2,1692.0,충청남도 서천군 서천읍 군사리 60-5,NaN,NaN
9965,(신청사) 소공원,5189.0,충청남도 서천군 서천읍 군사리 51-8,NaN,NaN
10086,희망,1495.8,서울특별시 송파구 문정동 103,NaN,37.485786
10104,잠실미성크로바,3644.0,서울특별시 송파구 신천동 17-6 일원,NaN,37.518732
10108,거여1(거마재정비),1139.7,서울특별시 송파구 거여동 181-143 일대,NaN,37.496441
10117,아랑,1575.3,서울특별시 송파구 송파동 44,NaN,37.508126
10137,큰솔,1500.0,서울특별시 송파구 오금동 47,NaN,37.504086
10140,한림,1504.0,서울특별시 송파구 장지동 834-2,NaN,37.485466
10143,호순이,1560.2,서울특별시 송파구 문정동 150-3,NaN,37.489845


In [16]:
gp[gp['위도'].isnull()]

,공원명,공원면적,소재지지번주소,경도,위도
9963,(신청사)어린이공원 2,1692.0,충청남도 서천군 서천읍 군사리 60-5,NaN,NaN
9965,(신청사) 소공원,5189.0,충청남도 서천군 서천읍 군사리 51-8,NaN,NaN
10073,위례별(위례 근린공원7호),12204.7,서울특별시 송파구 장지동 393임 일원,127.146847,NaN
10090,능안,1597.8,서울특별시 송파구 거여동 296-2,127.144058,NaN
10095,솔향(거마재정비),10042.0,서울특별시 송파구 마천동 199-3 일대,127.154217,NaN
10098,위례 근린공원12호,17872.0,서울특별시 송파구 장지동 산29임 일원,127.136184,NaN
10103,위례 어린이공원1호,1594.0,서울특별시 송파구 거여동 산71-14잡 일원,127.152664,NaN
10116,쌀섬,1646.7,서울특별시 송파구 잠실본동 319,127.082434,NaN
10364,널문,24128.0,"서울특별시 송파구 마천동 581,598",127.159264,NaN
10374,부리,12898.9,서울특별시 송파구 잠실동 22-5,127.087117,NaN


In [17]:
# 결측치 제거
gp.dropna(subset=['경도','위도'], inplace=True)
gp.isna().sum() 

공원명        0
공원면적       0
소재지지번주소    0
경도         0
위도         0
dtype: int64

In [18]:
type(gp.공원면적[0])

numpy.float64

In [19]:
gp.head()

,공원명,공원면적,소재지지번주소,경도,위도
0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
23,강변공원,1766.0,서울특별시 용산구 이촌동 408,126.975039,37.521338
24,솔밭어린이공원,2090.0,서울특별시 용산구 한강로1가 30-3,126.973668,37.537229
25,신창동어린이공원,257.0,서울특별시 용산구 신창동 100,126.954145,37.536698
26,서빙고공원,11879.0,서울특별시 용산구 용산동5가 24-2,126.971283,37.524782


In [20]:
gp.reset_index(inplace=True)
gp.head()

,index,공원명,공원면적,소재지지번주소,경도,위도
0,0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,23,강변공원,1766.0,서울특별시 용산구 이촌동 408,126.975039,37.521338
2,24,솔밭어린이공원,2090.0,서울특별시 용산구 한강로1가 30-3,126.973668,37.537229
3,25,신창동어린이공원,257.0,서울특별시 용산구 신창동 100,126.954145,37.536698
4,26,서빙고공원,11879.0,서울특별시 용산구 용산동5가 24-2,126.971283,37.524782


In [21]:
gp.columns

Index(['index', '공원명', '공원면적', '소재지지번주소', '경도', '위도'], dtype='object')

In [22]:
del gp['index']
gp

,공원명,공원면적,소재지지번주소,경도,위도
0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,강변공원,1766.0,서울특별시 용산구 이촌동 408,126.975039,37.521338
2,솔밭어린이공원,2090.0,서울특별시 용산구 한강로1가 30-3,126.973668,37.537229
3,신창동어린이공원,257.0,서울특별시 용산구 신창동 100,126.954145,37.536698
4,서빙고공원,11879.0,서울특별시 용산구 용산동5가 24-2,126.971283,37.524782
...,...,...,...,...,...
17309,방배,210517.2,서울특별시 서초구 방배동 686-201,126.995298,37.474225
17310,말죽거리,280822.6,서울특별시 서초구 서초동 산28,127.030469,37.480885
17311,서리풀,465938.3,서울특별시 서초구 방배동 산23-1,127.000420,37.484654
17312,방배,74847.6,서울특별시 서초구 방배동 산 43-7,126.989600,37.470535


In [23]:
gp.공원면적 = gp.공원면적.astype(int)

In [29]:
gp['소재지지번주소'][0]

'서울특별시 용산구 효창동 287-6'

In [32]:
gdf = gp[gp.소재지지번주소.str.contains('경기도')]
gdf.head()

,공원명,공원면적,소재지지번주소,경도,위도
190,꿈나무공원,2526,경기도 이천시 중리동 15,127.438819,37.278561
191,남천공원,3830,경기도 이천시 중리동 210,127.444436,37.278010
192,향교공원,1757,경기도 이천시 창전동 408,127.437589,37.284902
193,개나리공원,1650,경기도 이천시 창전동 414-13,127.439477,37.283860
194,달맞이공원,1650,경기도 이천시 창전동 423,127.443215,37.285854


In [ ]:
gadd_list = []
for add in gdf.소재지지번주소:
    if add.find('구 ') >= 0:
        gadd_list.append(''.join(add.split()[1:3]))
        

In [51]:
gdf['자치구'] = gdf.소재지지번주소.apply(lambda x: ''.join(x.split()[1]))
gdf.head()

,공원명,공원면적,소재지지번주소,경도,위도,자치구
190,꿈나무공원,2526,경기도 이천시 중리동 15,127.438819,37.278561,이천시
191,남천공원,3830,경기도 이천시 중리동 210,127.444436,37.278010,이천시
192,향교공원,1757,경기도 이천시 창전동 408,127.437589,37.284902,이천시
193,개나리공원,1650,경기도 이천시 창전동 414-13,127.439477,37.283860,이천시
194,달맞이공원,1650,경기도 이천시 창전동 423,127.443215,37.285854,이천시


In [39]:
gdf1 = gdf.groupby(['자치구'])[['공원면적']].agg(['sum'])


In [41]:
gdf1.columns = ['공원면적']
gdf1

,공원면적
자치구,
가평군,511673
고양시,6932749
과천시,315666
광명시,3397531
광주시,786478
구리시,912752
군포시,1455375
김포시,5792950
남양주시,4770408


In [42]:
file = 'data/경기도_geo_simple.json'
geo_data = json.load(open(file, encoding='utf-8'))

In [48]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']: # 경도,위도 표시
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]  # 경도,위도 표시
    return gu_dict

In [44]:
gu_dict = get_text_location(geo_data) # /경기도_geo_simple
gu_dict

{'양평군': [37.524478319941124, 127.57566999921767],
 '가평군': [37.80580217057866, 127.46168954149552],
 '연천군': [38.089173518920326, 126.98309880050942],
 '여주시': [37.30448255424536, 127.59166773209084],
 '포천시': [37.96989171146047, 127.23835163385141],
 '양주시': [37.79323667275805, 126.99994734085243],
 '광주시': [37.398346843618, 127.2921094056205],
 '화성시': [37.1612546033386, 126.88516922146444],
 '김포시': [37.667693477291564, 126.63558015047269],
 '안성시': [37.037984261825656, 127.31341594503061],
 '이천시': [37.198345154907365, 127.48867855904936],
 '파주시': [37.86614979102598, 126.8151997375004],
 '용인시수지구': [37.33143354615715, 127.07814257535223],
 '용인시기흥구': [37.26775157395476, 127.114890702286],
 '용인시처인구': [37.19555064723713, 127.26846597201227],
 '하남시': [37.51884197502213, 127.20638379320309],
 '의왕시': [37.35860200565407, 126.99342804765003],
 '군포시': [37.34247803452407, 126.92866515577917],
 '시흥시': [37.38683244726705, 126.79291699598996],
 '오산시': [37.15601650974868, 127.06212082657765],
 '남양주시': [37.

In [46]:
map = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=gdf1.공원면적, # 단계구분도로 보여줄 데이터
    columns=[gdf1.index, gdf1.공원면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='properties.name' # 지도에서 join할 항목
).add_to(map)
for gu_name in gdf1.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">경기도 시군별 공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map

KeyError: '고양시'

In [ ]:
map = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=gp.공원면적, # 단계구분도로 보여줄 데이터
    columns=[gp.index, gp.공원면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='properties.name' # 지도에서 join할 항목
).add_to(map)
for gu_name in gp.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">경기도 시군별 공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map